# Guide Complet : Fine-tuning d'un LLM Open-Source en Local

Ce notebook vous guide à travers le processus complet de fine-tuning d'un modèle de langage (LLM) open-source sur votre machine locale. Nous utiliserons des techniques modernes comme LoRA (Low-Rank Adaptation) pour un entraînement efficace.

## 🎯 Objectifs
- Apprendre à fine-tuner un LLM localement
- Utiliser LoRA pour un entraînement efficace en mémoire
- Préparer et traiter des données d'entraînement
- Évaluer et sauvegarder le modèle fine-tuné

## 📋 Prérequis
- Python 3.8+
- GPU recommandé (mais CPU possible)
- Au moins 8GB de RAM (16GB+ recommandé)
- Espace disque suffisant pour le modèle (~3-7GB selon le modèle)

## 🚀 Commençons !

## 1. 📦 Installation des Bibliothèques Nécessaires

Nous commençons par installer toutes les bibliothèques requises pour le fine-tuning. Cette étape peut prendre quelques minutes selon votre connexion internet.

In [ ]:
# Installation des bibliothèques principales
# Décommentez la ligne suivante si vous n'avez pas encore installé les dépendances
# !pip install -r ../requirements.txt

# Imports nécessaires
import torch
import os
import json
import pandas as pd
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    TrainingArguments, 
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import Dataset, DatasetDict
from peft import LoraConfig, get_peft_model, TaskType
import numpy as np
from datetime import datetime

print("✅ Bibliothèques importées avec succès")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🔧 Device disponible: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 Mémoire GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2. 🤖 Charger un Modèle LLM Open-Source

Nous allons charger un modèle open-source adapté au fine-tuning. Pour cet exemple, nous utiliserons Microsoft DialoGPT, un modèle conversationnel relativement léger et efficace.

**Options de modèles populaires :**
- `microsoft/DialoGPT-medium` (117M paramètres) - Bon pour débuter
- `microsoft/DialoGPT-large` (345M paramètres) - Plus puissant
- `facebook/opt-350m` (350M paramètres) - Alternative intéressante
- `EleutherAI/gpt-neo-125M` (125M paramètres) - Très léger pour tests

In [ ]:
# Configuration du modèle
MODEL_NAME = "microsoft/DialoGPT-medium"
CACHE_DIR = "../models/cache"

# Créer le répertoire de cache
os.makedirs(CACHE_DIR, exist_ok=True)

print(f"🔄 Chargement du modèle: {MODEL_NAME}")

# Charger le tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME, 
    cache_dir=CACHE_DIR,
    padding_side='left'  # Important pour les modèles causaux
)

# Ajouter un token de padding si nécessaire
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"✅ Tokenizer chargé")
print(f"📝 Taille du vocabulaire: {len(tokenizer):,} tokens")
print(f"🔚 Token EOS: '{tokenizer.eos_token}' (ID: {tokenizer.eos_token_id})")
print(f"📋 Token PAD: '{tokenizer.pad_token}' (ID: {tokenizer.pad_token_id})")

# Charger le modèle
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    cache_dir=CACHE_DIR,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

if not torch.cuda.is_available():
    model = model.to(device)

print(f"✅ Modèle chargé sur {device}")

# Informations sur le modèle
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"📊 Paramètres totaux: {total_params:,}")
print(f"🎯 Paramètres entraînables: {trainable_params:,}")
print(f"📈 Pourcentage entraînable: {(trainable_params/total_params)*100:.1f}%")

## 3. 📊 Préparer les Données d'Entraînement

Nous allons créer un jeu de données d'exemple pour le fine-tuning. Dans un cas réel, vous remplaceriez ceci par vos propres données.

**Format des données pour la conversation :**
```json
{
  "conversations": [
    {
      "input": "Question de l'utilisateur",
      "output": "Réponse de l'assistant"
    }
  ]
}
```

In [ ]:
# Créer des données d'exemple pour la démonstration
# Dans un cas réel, vous chargeriez vos propres données

sample_conversations = [
    {
        "input": "Bonjour, comment allez-vous ?",
        "output": "Bonjour ! Je vais très bien, merci de demander. Comment puis-je vous aider aujourd'hui ?"
    },
    {
        "input": "Pouvez-vous m'expliquer ce qu'est l'intelligence artificielle ?",
        "output": "L'intelligence artificielle (IA) est un domaine de l'informatique qui vise à créer des machines capables de réaliser des tâches nécessitant normalement l'intelligence humaine, comme la compréhension du langage, la reconnaissance d'images, ou la prise de décisions."
    },
    {
        "input": "Quels sont les avantages du machine learning ?",
        "output": "Le machine learning offre plusieurs avantages : automatisation des tâches complexes, capacité à traiter de grandes quantités de données, amélioration continue des performances, détection de patterns invisibles à l'œil humain, et personnalisation des expériences utilisateur."
    },
    {
        "input": "Comment fonctionne un réseau de neurones ?",
        "output": "Un réseau de neurones est inspiré du cerveau humain. Il est composé de neurones artificiels organisés en couches qui traitent l'information. Chaque connexion a un poids qui s'ajuste pendant l'entraînement pour apprendre à reconnaître des patterns dans les données."
    },
    {
        "input": "Qu'est-ce que le deep learning ?",
        "output": "Le deep learning est une sous-branche du machine learning qui utilise des réseaux de neurones profonds (avec plusieurs couches cachées) pour apprendre des représentations complexes des données. Il excelle dans des tâches comme la vision par ordinateur et le traitement du langage naturel."
    },
    # Ajout de plus d'exemples pour un entraînement plus robuste
    {
        "input": "Merci pour votre aide !",
        "output": "Je vous en prie ! C'était un plaisir de vous aider. N'hésitez pas si vous avez d'autres questions."
    },
    {
        "input": "Qu'est-ce que Python ?",
        "output": "Python est un langage de programmation populaire, connu pour sa simplicité et sa lisibilité. Il est largement utilisé en data science, intelligence artificielle, développement web et automatisation."
    },
    {
        "input": "Comment apprendre la programmation ?",
        "output": "Pour apprendre la programmation, commencez par choisir un langage (Python est excellent pour débuter), pratiquez régulièrement avec des petits projets, utilisez des ressources en ligne, et n'hésitez pas à rejoindre des communautés de développeurs."
    }
]

print(f"📚 Dataset créé avec {len(sample_conversations)} exemples")

# Afficher quelques exemples
print("\n🔍 Aperçu des données:")
for i, conv in enumerate(sample_conversations[:3]):
    print(f"\nExemple {i+1}:")
    print(f"  Input: {conv['input']}")
    print(f"  Output: {conv['output'][:100]}...")

In [ ]:
# Fonction pour formater les conversations
def format_conversation(example):
    """Formate une conversation pour l'entraînement"""
    input_text = example["input"]
    output_text = example["output"]
    
    # Format conversationnel avec tokens spéciaux
    formatted_text = f"<|user|>{input_text}<|endoftext|><|assistant|>{output_text}<|endoftext|>"
    return {"text": formatted_text}

# Convertir en Dataset HuggingFace
dataset = Dataset.from_list(sample_conversations)
dataset = dataset.map(format_conversation)

print("✅ Dataset formaté")
print(f"📊 Colonnes: {dataset.column_names}")
print(f"📏 Taille: {len(dataset)} exemples")

# Exemple de texte formaté
print(f"\n📝 Exemple de texte formaté:")
print(dataset[0]["text"])

# Diviser en train/validation (80/20)
dataset_split = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset_split["train"]
eval_dataset = dataset_split["test"]

print(f"\n📊 Split des données:")
print(f"  Train: {len(train_dataset)} exemples")
print(f"  Validation: {len(eval_dataset)} exemples")

# Fonction de tokenisation
def tokenize_function(examples):
    """Tokenise les textes pour l'entraînement"""
    # Tokeniser avec padding et truncation
    result = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors=None
    )
    
    # Pour l'entraînement causal, les labels sont les mêmes que input_ids
    result["labels"] = result["input_ids"].copy()
    
    return result

# Tokeniser les datasets
train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["input", "output", "text"]
)

eval_dataset = eval_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["input", "output", "text"]
)

print("✅ Datasets tokenisés")
print(f"🔧 Colonnes finales: {train_dataset.column_names}")

# Vérifier la forme des données
print(f"📐 Forme d'un échantillon:")
sample = train_dataset[0]
for key, value in sample.items():
    print(f"  {key}: {len(value) if isinstance(value, list) else type(value)}")

## 4. ⚙️ Configurer le Fine-tuning avec LoRA

LoRA (Low-Rank Adaptation) est une technique qui permet de fine-tuner efficacement de gros modèles en n'entraînant qu'une petite fraction des paramètres. Cela réduit considérablement les besoins en mémoire et en calcul.

**Avantages de LoRA :**
- 🚀 Entraînement plus rapide
- 💾 Moins d'utilisation mémoire
- 🎯 Résultats comparables au fine-tuning complet
- 💽 Modèles plus petits à stocker

In [ ]:
# Configuration LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # Type de tâche: modèle de langage causal
    r=16,                          # Rang de la décomposition (plus bas = moins de paramètres)
    lora_alpha=32,                 # Paramètre de scaling LoRA
    lora_dropout=0.1,              # Dropout pour la régularisation
    target_modules=["c_attn", "c_proj", "c_fc"],  # Modules à adapter (spécifique au modèle)
    bias="none"                    # Ne pas adapter les bias
)

print("🔧 Configuration LoRA:")
print(f"  Rang (r): {lora_config.r}")
print(f"  Alpha: {lora_config.lora_alpha}")
print(f"  Dropout: {lora_config.lora_dropout}")
print(f"  Modules cibles: {lora_config.target_modules}")

# Appliquer LoRA au modèle
model = get_peft_model(model, lora_config)

# Afficher les informations après LoRA
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n📊 Paramètres après LoRA:")
print(f"  Total: {total_params:,}")
print(f"  Entraînables: {trainable_params:,}")
print(f"  Pourcentage entraînable: {(trainable_params/total_params)*100:.2f}%")
print(f"  Réduction: {((total_params-trainable_params)/total_params)*100:.1f}% de paramètres en moins à entraîner!")

# Vérifier que LoRA est bien appliqué
model.print_trainable_parameters()

In [ ]:
# Configuration des paramètres d'entraînement
output_dir = "../models/finetuned"
os.makedirs(output_dir, exist_ok=True)

training_args = TrainingArguments(
    output_dir=output_dir,
    
    # Paramètres d'entraînement
    num_train_epochs=3,                    # Nombre d'époques
    per_device_train_batch_size=2,         # Taille de batch (ajustez selon votre GPU)
    per_device_eval_batch_size=2,          # Taille de batch pour l'évaluation
    gradient_accumulation_steps=4,         # Accumulation de gradients
    
    # Optimiseur
    learning_rate=2e-4,                    # Taux d'apprentissage
    weight_decay=0.01,                     # Décroissance des poids
    warmup_ratio=0.1,                      # Warmup
    lr_scheduler_type="cosine",            # Type de scheduler
    
    # Sauvegarde et évaluation
    save_strategy="steps",
    save_steps=50,                         # Sauvegarder tous les 50 steps
    evaluation_strategy="steps",
    eval_steps=25,                         # Évaluer tous les 25 steps
    save_total_limit=3,                    # Garder seulement 3 checkpoints
    load_best_model_at_end=True,
    
    # Logging
    logging_steps=10,
    logging_dir="../logs",
    
    # Optimisations
    fp16=torch.cuda.is_available(),        # Précision mixte si GPU disponible
    dataloader_num_workers=2,
    remove_unused_columns=False,
    
    # Autres
    seed=42,
    data_seed=42,
    report_to=[],                          # Désactiver W&B pour la démo
)

print("✅ Arguments d'entraînement configurés")
print(f"🎯 Époques: {training_args.num_train_epochs}")
print(f"📦 Batch size: {training_args.per_device_train_batch_size}")
print(f"🎚️ Learning rate: {training_args.learning_rate}")
print(f"💾 Répertoire de sortie: {training_args.output_dir}")
print(f"🔥 FP16: {training_args.fp16}")

# Data collator pour l'entraînement de modèles de langage
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Pas de masquage pour les modèles causaux
    pad_to_multiple_of=8 if training_args.fp16 else None
)

print("✅ Data collator configuré")

## 5. 🚀 Lancer le Fine-tuning

Maintenant que tout est configuré, nous pouvons lancer l'entraînement ! Le processus peut prendre quelques minutes selon votre matériel.

**⏱️ Temps estimé :**
- CPU: 10-30 minutes
- GPU (GTX 1060/RTX 2060): 5-15 minutes  
- GPU (RTX 3080+): 2-5 minutes

In [ ]:
# Créer le trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("✅ Trainer créé")
print("\n🔥 Début de l'entraînement...")
print("="*50)

# Lancer l'entraînement
start_time = datetime.now()

try:
    # Entraîner le modèle
    trainer.train()
    
    end_time = datetime.now()
    training_time = end_time - start_time
    
    print("="*50)
    print("🎉 Entraînement terminé avec succès !")
    print(f"⏱️ Temps total: {training_time}")
    
    # Afficher les métriques finales
    final_metrics = trainer.state.log_history[-1]
    print(f"\n📊 Métriques finales:")
    for key, value in final_metrics.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.4f}")
        else:
            print(f"  {key}: {value}")
            
except Exception as e:
    print(f"❌ Erreur durante l'entraînement: {e}")
    raise

## 6. 📈 Évaluer le Modèle Fine-tuné

Maintenant que l'entraînement est terminé, évaluons les performances du modèle et testons-le avec quelques exemples.

In [ ]:
# Évaluation sur le jeu de validation
print("🔍 Évaluation sur le jeu de validation...")
eval_results = trainer.evaluate()

print("📊 Résultats d'évaluation:")
for metric, value in eval_results.items():
    if isinstance(value, float):
        print(f"  {metric}: {value:.4f}")
    else:
        print(f"  {metric}: {value}")

# Fonction pour générer du texte avec le modèle fine-tuné
def generate_response(prompt, max_length=200, temperature=0.7):
    """Génère une réponse avec le modèle fine-tuné"""
    # Formater le prompt
    formatted_prompt = f"<|user|>{prompt}<|endoftext|><|assistant|>"
    
    # Tokeniser
    inputs = tokenizer.encode(formatted_prompt, return_tensors="pt").to(device)
    
    # Générer
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=len(inputs[0]) + max_length,
            temperature=temperature,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    # Décoder seulement la partie générée
    generated_text = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
    
    # Nettoyer la réponse
    if "<|endoftext|>" in generated_text:
        generated_text = generated_text.split("<|endoftext|>")[0]
    
    return generated_text.strip()

# Tester le modèle avec quelques exemples
test_prompts = [
    "Bonjour, pouvez-vous m'aider ?",
    "Qu'est-ce que le machine learning ?",
    "Comment puis-je apprendre l'IA ?",
    "Expliquez-moi Python simplement",
    "Merci pour votre aide !"
]

print("\n🧪 Test du modèle fine-tuné:")
print("="*60)

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n📝 Test {i}:")
    print(f"👤 Utilisateur: {prompt}")
    
    response = generate_response(prompt)
    print(f"🤖 Assistant: {response}")
    print("-" * 40)

## 7. 💾 Sauvegarder et Charger le Modèle Fine-tuné

Une fois satisfait des résultats, nous pouvons sauvegarder le modèle pour une utilisation future.

In [ ]:
# Sauvegarder le modèle fine-tuné
save_path = "../models/my_finetuned_model"
os.makedirs(save_path, exist_ok=True)

print(f"💾 Sauvegarde du modèle dans {save_path}...")

# Sauvegarder le modèle LoRA
model.save_pretrained(save_path)

# Sauvegarder le tokenizer
tokenizer.save_pretrained(save_path)

# Sauvegarder les métriques d'entraînement
metrics_path = os.path.join(save_path, "training_metrics.json")
with open(metrics_path, "w") as f:
    json.dump(trainer.state.log_history, f, indent=2)

print("✅ Modèle sauvegardé avec succès !")

# Lister les fichiers sauvegardés
import os
saved_files = os.listdir(save_path)
print(f"\n📁 Fichiers sauvegardés:")
for file in saved_files:
    file_path = os.path.join(save_path, file)
    if os.path.isfile(file_path):
        size = os.path.getsize(file_path) / (1024*1024)  # MB
        print(f"  📄 {file} ({size:.1f} MB)")

print(f"\n📊 Taille totale du modèle:")
total_size = sum(os.path.getsize(os.path.join(save_path, f)) 
                for f in saved_files if os.path.isfile(os.path.join(save_path, f)))
print(f"  {total_size / (1024*1024):.1f} MB")

In [ ]:
# Démonstration du rechargement du modèle
print("🔄 Démonstration du rechargement du modèle...")

# Pour recharger le modèle plus tard, utilisez ce code:
from peft import PeftModel

def load_finetuned_model(model_path, base_model_name):
    """Charge un modèle fine-tuné avec LoRA"""
    
    # Charger le tokenizer
    loaded_tokenizer = AutoTokenizer.from_pretrained(model_path)
    
    # Charger le modèle de base
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None
    )
    
    # Charger les adaptateurs LoRA
    loaded_model = PeftModel.from_pretrained(base_model, model_path)
    
    return loaded_model, loaded_tokenizer

# Exemple d'utilisation (commenté pour éviter de recharger maintenant)
"""
loaded_model, loaded_tokenizer = load_finetuned_model(
    save_path, 
    MODEL_NAME
)
print("✅ Modèle rechargé avec succès !")
"""

print("📝 Code de rechargement prêt à utiliser !")

# Afficher le code d'utilisation
usage_code = f'''
# Pour utiliser votre modèle fine-tuné plus tard:

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# Chemins
model_path = "{save_path}"
base_model_name = "{MODEL_NAME}"

# Charger le tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Charger le modèle de base
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Charger les adaptateurs LoRA
model = PeftModel.from_pretrained(base_model, model_path)

# Fonction de génération
def chat_with_model(prompt):
    formatted_prompt = f"<|user|>{{prompt}}<|endoftext|><|assistant|>"
    inputs = tokenizer.encode(formatted_prompt, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=len(inputs[0]) + 200,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )
    
    response = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
    return response.split("<|endoftext|>")[0].strip()

# Utilisation
response = chat_with_model("Bonjour, comment allez-vous ?")
print(response)
'''

print("💡 Code d'utilisation:")
print(usage_code)

## 🎉 Conclusion

Félicitations ! Vous avez réussi à fine-tuner un LLM en local avec LoRA. 

### 🎯 Ce que vous avez appris :
- ✅ Charger et préparer un modèle LLM open-source
- ✅ Préparer et formater des données d'entraînement
- ✅ Configurer LoRA pour un fine-tuning efficace
- ✅ Entraîner le modèle avec des paramètres optimaux
- ✅ Évaluer et tester le modèle fine-tuné
- ✅ Sauvegarder et recharger le modèle

### 🚀 Prochaines étapes :
1. **Données réelles** : Remplacez les données d'exemple par vos propres données
2. **Modèles plus gros** : Essayez des modèles plus grands comme Llama 2 7B
3. **Optimisations** : Expérimentez avec différents paramètres LoRA
4. **Déploiement** : Intégrez votre modèle dans une application
5. **Évaluation avancée** : Utilisez des métriques plus sophistiquées

### 📚 Ressources utiles :
- [Documentation PEFT](https://huggingface.co/docs/peft/)
- [LoRA Paper](https://arxiv.org/abs/2106.09685)
- [Transformers Documentation](https://huggingface.co/docs/transformers/)
- [Datasets Documentation](https://huggingface.co/docs/datasets/)

### 💡 Conseils pour l'optimisation :
- **Augmentez les données** : Plus de données = meilleur modèle
- **Ajustez LoRA** : Expérimentez avec `r` et `alpha`
- **Monitoring** : Utilisez W&B ou TensorBoard pour suivre l'entraînement
- **GPU** : Un GPU accélère considérablement le processus

Bonne chance avec vos projets de fine-tuning ! 🚀